In [1]:
# Load and formate file with SNP Betas
betaFile = open("emerge-prs-risk-beta.csv", "r")
betaFile = [x.split(",") for x in betaFile.readlines()]
# Save the header for metadata
betaHeader = betaFile[0]
betaHeader[-1] = betaHeader[-1].split("\n")[0]
betaFile = betaFile[1:]

# Remove the newline characters from the end of each SNP
for snp in betaFile:
    snp[-1] = snp[-1].split("\n")[0]

print(betaFile[0:10])

[['1', '22587728', '1:22587728_T/C', 'rs72647484', 'C', 'T', '0', '0.0504'], ['1', '38455891', '1:38455891_G/C', 'rs4360494', 'C', 'G', '0', '0.0379'], ['1', '55246035', '1:55246035_T/C', 'rs12144319', 'C', 'T', '1', '0.0661'], ['1', '62673037', '1:62673037_T/C', 'rs7542665', 'T', 'C', '0', '0.033423755'], ['1', '183002639', '1:183002639_A/G', 'rs6678517', 'G', 'A', '0', '0.073'], ['1', '222112634', '1:222112634_A/G', 'rs17011141', 'G', 'A', '1', '0.0877'], ['1', '245181421', '1:245181421_T/C', 'rs201395236', 'C', 'T', '0', '0.243038049'], ['2', '48686695', '2:48686695_T/A', 'rs7606562', 'A', 'T', '0', '0.041392685'], ['2', '98275354', '2:98275354_G/A', 'rs11692435', 'A', 'G', '0', '0.049218023'], ['2', '159964552', '2:159964552_T/C', 'rs448513', 'C', 'T', '1', '0.0054']]


In [2]:
# Load and Format Individual Variant File
variantFile = open("concatenated_merged_variants.raw", "r")
variantFile = [x.split(" ") for x in variantFile.readlines()]
# Save the header for metadata
variantHeader = variantFile[0]
variantHeader[-1] = variantHeader[-1].split("\n")[0]
variantFile = variantFile[1:]

# Remove the newline characters from the end of each individual
for ind in variantFile:
    ind[-1] = ind[-1].split("\n")[0]
    
print(variantFile[0:10])

[['68075207', '68075207', '0', '0', '0', '-9', '0', '0', '0', '2', '1', '1', '0', '0', '1', '2', '1', '2', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0', '2', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '0', '2', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '2', '0', '0', '2', '1', '0', '2', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '0', '2', '0', '2', '1', '0', '1', '1', '0', '0', '1', '1', '1', '2', '2', '0', '2', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0', '0', '0', '0', '2', '1', '2', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '2', '0', '1', '1', '1', '2', '1', '2'], ['68914701', '68914701', '0', '0', '0', '-9', '0', '2', '1', '0', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '2', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1', '1', '1', '1', '1', '0', '1', '

In [3]:
# Create a dictionary that maps SNP "name" (Chr#:Pos) to SNP information (containing risk allele, beta, chromosome info, etc.)
betaDict = dict()
for snp in betaFile:
    snpName = snp[2].split("_")[0]
    betaDict[snpName] = snp

# Example output of SNP 1:22587728
print(betaDict["1:22587728"])

['1', '22587728', '1:22587728_T/C', 'rs72647484', 'C', 'T', '0', '0.0504']


In [4]:
# In variantFile, individual SNP begins from array index 6 onwards and contains 140 SNPs
print(len(variantHeader[6:]))

140


In [5]:
# Create a dictionary that maps SNP "name" (Chr#:Pos) to array index value from variantFile and allele counted ("A", "C", "G", or "T")
count = 6
headerDict = dict()
for snp in variantHeader[6:]:
    snpName = snp.split("_")[0]
    alleleName = snp.split("_")[1]
    headerDict[snpName] = [count, alleleName]
    count = count + 1
    
# Example output of SNP 1:22587728
print(headerDict["1:22587728"])

[6, 'C']


In [6]:
# Creata a dictionary that maps individuals to a list containing SNPs and the number of risk alleles for that SNP that individual has
indDict = dict()
for ind in variantFile:
    indName = ind[1]
    snpList = list()
    for snp in betaDict.keys():
        riskAllele = betaDict[snp][4] if betaDict[snp][6] == "1" else betaDict[snp][5]
        headerColumn = headerDict[snp][0]
        observedAllele = headerDict[snp][1]
        riskAlleleCount = int(ind[int(headerColumn)]) if riskAllele == observedAllele else (2 - int(ind[int(headerColumn)]))
        snpList.append((snp, riskAlleleCount))
    indDict[indName] = snpList

# Example output of individual #68075207
print(indDict["68075207"])

[('1:22587728', 2), ('1:38455891', 2), ('1:55246035', 0), ('1:62673037', 0), ('1:183002639', 1), ('1:222112634', 1), ('1:245181421', 2), ('2:48686695', 0), ('2:98275354', 2), ('2:159964552', 1), ('2:199612407', 1), ('2:199781586', 0), ('2:219191256', 2), ('3:40915239', 1), ('3:53088285', 1), ('3:66365163', 1), ('3:112903888', 0), ('3:112916918', 0), ('3:112999560', 2), ('3:133701119', 0), ('3:133748789', 2), ('3:169517436', 2), ('4:94938618', 0), ('4:106128760', 2), ('4:145659064', 0), ('5:1240204', 1), ('5:1296486', 1), ('5:40102443', 1), ('5:40280076', 1), ('5:98206082', 2), ('5:112097351', 0), ('5:125988175', 2), ('5:134467220', 2), ('6:12292772', 2), ('6:29809860', 1), ('6:30758466', 2), ('6:31010185', 0), ('6:31315512', 2), ('6:31449620', 2), ('6:32191339', 0), ('6:32593080', 2), ('6:35569562', 1), ('6:36623379', 0), ('6:41702582', 0), ('6:55566108', 0), ('6:55712124', 2), ('7:45136423', 2), ('7:46094089', 1), ('7:46926695', 1), ('7:47511161', 2), ('8:117630683', 0), ('8:117632965

In [12]:
# Generate PRS using formula: PRS = Sum over all SNPS of snpBETA * riskAlleleCount
# Output PRS to separate file
# Create dictionary mapping individual to their risk score

output = open("prsColon.txt", "w")
output.write("EMERGE ID, PRS\n")

prsDict = dict()

for ind in indDict.keys():
    runningTotal = 0
    snpContributions = list()
    for snp in indDict[ind]:
        snpBeta = float(betaDict[snp[0]][7])
        if betaDict[snp[0]][6] == '1':
            runningTotal += snp[1] * snpBeta
            snpContributions.append(snp[1] * snpBeta)
        else:
            runningTotal += (2 - snp[1]) * -snpBeta
            snpContributions.append((2 - snp[1]) * -snpBeta)
    prsDict[ind] = runningTotal, snpContributions
    output.write(ind + "," + str(runningTotal) + "\n")

# Example output of individual #68075207
print(sum(prsDict["68075207"][1]))

-0.871995546


In [13]:
# Load and format Colon Cancer status file
colonFile = open("emerge-crc-status-20190-06-06.txt", "r")
colonFile = [x.split("\t") for x in colonFile.readlines()]
# Save the header for metadata
colonHeader = colonFile[0]
colonHeader[-1] = colonHeader[-1].split("\n")[0]
colonFile = colonFile[1:]

for ind in colonFile:
    ind[-1] = ind[-1].split("\n")[0]

print(colonFile[0:10])

[['42000240', 'colu', 'M', '1931', '0', '83', 'NA', '0', '0', 'C17998', 'C41221', 'C68751', '77', '1808'], ['42000306', 'colu', 'M', '1982', '0', 'NA', 'NA', '0', '0', 'C41261', 'C41222', 'C68751', '31', '3'], ['42000697', 'colu', 'F', '1951', '0', 'NA', 'NA', '0', '0', 'C41261', 'C41221', 'C67151', '0', '0'], ['42001430', 'colu', 'F', '1951', '0', 'NA', 'NA', '0', '0', 'C41261', 'C41222', 'C65108', '61', '1'], ['42001533', 'colu', 'M', '1960', '0', 'NA', 'NA', '0', '0', 'C41261', 'C41222', 'C67151', '47', '14'], ['42003615', 'colu', 'M', '1954', '0', 'NA', 'NA', '0', '0', 'C17998', 'C41221', 'C67151', '0', '0'], ['42004363', 'colu', 'F', '1964', '0', '49', 'NA', '0', '0', 'C41260', 'C41222', 'C65108', '0', '0'], ['42004587', 'colu', 'M', '1928', '0', 'NA', 'NA', '0', '0', 'C41261', 'C17459', 'C67151', '81', '18'], ['42005489', 'colu', 'F', '1981', '0', 'NA', 'NA', '0', '0', 'C17998', 'C41221', 'C67151', '0', '0'], ['42005962', 'colu', 'F', '1928', '0', '89', 'NA', '0', '0', 'C17998', 

In [14]:
# Create dictionary mapping individual to colon cancer status

colonDict = dict()
for ind in colonFile:
    indName = ind[0]
    cancerStatus = ind[4]
    colonDict[indName] = cancerStatus

print(len(colonDict.keys()))
# Example output for ind #42000240
print(colonDict["42000240"])

50850
0


In [27]:
# Generate training data
features = list()
labels = list()
count = 0

for ind in colonDict.keys():
    if colonDict[ind] == '0' and (ind in prsDict.keys()) and count < 5000:
        features.append(prsDict[ind][1])
        labels.append(0)
        count += 1
    elif colonDict[ind] == '1' and (ind in prsDict.keys()):
        features.append(prsDict[ind][1])
        labels.append(1)
        features.append(prsDict[ind][1])
        labels.append(1)

print(len(features))
print(len(labels))

5938
5938


In [36]:
# Generate numpy arrays from training data
import numpy as np
X = np.asarray(features)
y = np.asarray(labels)

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42, stratify = y)
mfc = MLPClassifier(alpha = 1, max_iter=1000, hidden_layer_sizes=(128,64,1))
# mfc = KNeighborsClassifier(4)
# mfc = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 10), n_estimators = 200)
# mfc = RandomForestClassifier(max_depth = 5, n_estimators = 10, max_features = 1)
mfc.fit(X_train, y_train)



MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(128, 64, 1), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [37]:
from sklearn.metrics import confusion_matrix, classification_report
pred_mfc = mfc.predict(X_test)
print(classification_report(y_test, pred_mfc))
print(confusion_matrix(y_test, pred_mfc))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      2001
           1       0.00      0.00      0.00       375

    accuracy                           0.84      2376
   macro avg       0.42      0.50      0.46      2376
weighted avg       0.71      0.84      0.77      2376

[[2001    0]
 [ 375    0]]


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
